In [ ]:
import subprocess, textwrap, os, sys

# Check GPU
print("=== NVIDIA SMI ===")
os.system("nvidia-smi || echo 'No GPU detected (or driver not accessible)'")


=== NVIDIA SMI ===


0

In [ ]:
import torch

print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Device count:", torch.cuda.device_count())
    print("Current device:", torch.cuda.current_device())
    print("Device name:", torch.cuda.get_device_name(0))


CUDA available: True
Device count: 1
Current device: 0
Device name: Tesla T4


In [ ]:
!git clone --branch get-statistics --single-branch https://github.com/Arpnik/EDU-Chunking-RAG.git
%cd /content/EDU-Chunking-RAG
!ls


Cloning into 'EDU-Chunking-RAG'...
remote: Enumerating objects: 363, done.
remote: Counting objects: 100% (363/363), done.
remote: Compressing objects: 100% (254/254), done.
remote: Total 363 (delta 174), reused 289 (delta 107), pack-reused 0 (from 0)
Receiving objects: 100% (363/363), 511.71 KiB | 2.36 MiB/s, done.
Resolving deltas: 100% (174/174), done.
/content/EDU-Chunking-RAG
com  dataset  docker-compose.yml  pyproject.toml  README.md


In [ ]:
!pip install uv
!uv venv .venv    # this creates /content/EDU-Chunking-RAG/.venv


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 122.5 MB/s eta 0:00:00
Using CPython 3.12.12 interpreter at: /usr/bin/python3
Creating virtual environment at: .venv
Activate with: source .venv/bin/activate


In [ ]:
%cd /content/EDU-Chunking-RAG
!uv pip install -e . -p .venv/bin/python


/content/EDU-Chunking-RAG
Resolved 68 packages in 723ms
Prepared 68 packages in 1m 12s
Installed 68 packages in 534ms
 + accelerate==1.12.0
 + annotated-types==0.7.0
 + anyio==4.11.0
 + certifi==2025.11.12
 + charset-normalizer==3.4.4
 + fever-rag==0.1.0 (from file:///content/EDU-Chunking-RAG)
 + filelock==3.20.0
 + fsspec==2025.10.0
 + grpcio==1.76.0
 + h11==0.16.0
 + h2==4.3.0
 + hf-xet==1.2.0
 + hpack==4.1.0
 + httpcore==1.0.9
 + httpx==0.25.2
 + huggingface-hub==0.36.0
 + hyperframe==6.1.0
 + idna==3.11
 + jinja2==3.1.6
 + joblib==1.5.2
 + markupsafe==3.0.3
 + mpmath==1.3.0
 + networkx==3.6
 + numpy==2.3.5
 + nvidia-cublas-cu12==12.8.4.1
 + nvidia-cuda-cupti-cu12==12.8.90
 + nvidia-cuda-nvrtc-cu12==12.8.93
 + nvidia-cuda-runtime-cu12==12.8.90
 + nvidia-cudnn-cu12==9.10.2.21
 + nvidia-cufft-cu12==11.3.3.83
 + nvidia-cufile-cu12==1.13.1.3
 + nvidia-curand-cu12==10.3.9.90
 + nvidia-cusolver-cu12==11.7.3.90
 + nvidia-cusparse-cu12==12.5.8.93
 + nvidia-cusparselt-cu12==0.7.1
 + nvidia-n

In [ ]:
from google.colab import files
uploaded = files.upload()    # upload e.g. fever_dataset.zip


Saving reduced_fever_data.zip to reduced_fever_data.zip


In [ ]:
!unzip reduced_fever_data.zip -d dataset
!ls dataset


Archive:  reduced_fever_data.zip
   creating: dataset/reduced_fever_data/
  inflating: dataset/__MACOSX/._reduced_fever_data  
  inflating: dataset/reduced_fever_data/train.jsonl  
  inflating: dataset/__MACOSX/reduced_fever_data/._train.jsonl  
  inflating: dataset/reduced_fever_data/paper_test.jsonl  
  inflating: dataset/__MACOSX/reduced_fever_data/._paper_test.jsonl  
   creating: dataset/reduced_fever_data/wiki/
  inflating: dataset/__MACOSX/reduced_fever_data/._wiki  
  inflating: dataset/reduced_fever_data/paper_dev.jsonl  
  inflating: dataset/__MACOSX/reduced_fever_data/._paper_dev.jsonl  
  inflating: dataset/reduced_fever_data/wiki/filtered_evidence.jsonl  
  inflating: dataset/__MACOSX/reduced_fever_data/wiki/._filtered_evidence.jsonl  
data_exploration.ipynb	__MACOSX  reduced_fever_data


In [ ]:
from google.colab import files
uploaded = files.upload()  # upload something like edu_segmenter_linear.zip


Saving MLPbert_grp6_dim_512_256_128_cw0_5_rank_16_alpha_128_ep10_noES.zip to MLPbert_grp6_dim_512_256_128_cw0_5_rank_16_alpha_128_ep10_noES.zip


In [ ]:
%cd /content/EDU-Chunking-RAG
!mkdir -p edu_segmenter_linear
!unzip -o MLPbert_grp6_dim_512_256_128_cw0_5_rank_16_alpha_128_ep10_noES.zip -d edu_segmenter_linear

/content/EDU-Chunking-RAG
Archive:  MLPbert_grp6_dim_512_256_128_cw0_5_rank_16_alpha_128_ep10_noES.zip
   creating: edu_segmenter_linear/MLPbert_grp6_dim_512_256_128_cw0_5_rank_16_alpha_128_ep10_noES/
   creating: edu_segmenter_linear/MLPbert_grp6_dim_512_256_128_cw0_5_rank_16_alpha_128_ep10_noES/checkpoint-468/
  inflating: edu_segmenter_linear/MLPbert_grp6_dim_512_256_128_cw0_5_rank_16_alpha_128_ep10_noES/checkpoint-468/vocab.txt  
  inflating: edu_segmenter_linear/MLPbert_grp6_dim_512_256_128_cw0_5_rank_16_alpha_128_ep10_noES/checkpoint-468/adapter_config.json  
  inflating: edu_segmenter_linear/MLPbert_grp6_dim_512_256_128_cw0_5_rank_16_alpha_128_ep10_noES/checkpoint-468/rng_state.pth  
  inflating: edu_segmenter_linear/MLPbert_grp6_dim_512_256_128_cw0_5_rank_16_alpha_128_ep10_noES/checkpoint-468/adapter_model.safetensors  
  inflating: edu_segmenter_linear/MLPbert_grp6_dim_512_256_128_cw0_5_rank_16_alpha_128_ep10_noES/checkpoint-468/tokenizer_config.json  
  inflating: edu_segment

In [ ]:
!ls edu_segmenter_linear

MLPbert_grp6_dim_512_256_128_cw0_5_rank_16_alpha_128_ep10_noES


In [ ]:
%cd /content/EDU-Chunking-RAG
!ls edu_segmenter_linear/MLPbert_grp6_dim_512_256_128_cw0_5_rank_16_alpha_128_ep10_noES

/content/EDU-Chunking-RAG
best_model	checkpoint-546	checkpoint-702
checkpoint-468	checkpoint-624	checkpoint-780


In [ ]:
MODEL_PATH = "edu_segmenter_linear/MLPbert_grp6_dim_512_256_128_cw0_5_rank_16_alpha_128_ep10_noES/best_model"

In [ ]:
from pathlib import Path

# Project root in Colab
PROJECT_ROOT = Path("/content/EDU-Chunking-RAG")

# Point to the reduced dataset inside the existing dataset/ folder
DATASET_ROOT = PROJECT_ROOT / "dataset" / "reduced_fever_data"
WIKI_DIR_PATH = DATASET_ROOT / "wiki"
CLAIM_FILE_PATH = DATASET_ROOT / "paper_dev.jsonl"

# EDU segmenter model path (adjust folder name if yours is different)
MODEL_PATH_PATH = PROJECT_ROOT / "edu_segmenter_linear/MLPbert_grp6_dim_512_256_128_cw0_5_rank_16_alpha_128_ep10_noES"  / "best_model"

# String versions for use in !python CLI commands
DATASET_ROOT_STR = str(DATASET_ROOT)
WIKI_DIR = str(WIKI_DIR_PATH)
CLAIM_FILE = str(CLAIM_FILE_PATH)
MODEL_PATH = str(MODEL_PATH_PATH)

print("DATASET_ROOT:", DATASET_ROOT_STR)
print("WIKI_DIR:", WIKI_DIR)
print("CLAIM_FILE:", CLAIM_FILE)
print("MODEL_PATH:", MODEL_PATH)

# Optional sanity check: see which ones actually exist
for p in [DATASET_ROOT, WIKI_DIR_PATH, CLAIM_FILE_PATH, MODEL_PATH_PATH]:
    print(p, "exists:", p.exists())


DATASET_ROOT: /content/EDU-Chunking-RAG/dataset/reduced_fever_data
WIKI_DIR: /content/EDU-Chunking-RAG/dataset/reduced_fever_data/wiki
CLAIM_FILE: /content/EDU-Chunking-RAG/dataset/reduced_fever_data/paper_dev.jsonl
MODEL_PATH: /content/EDU-Chunking-RAG/edu_segmenter_linear/MLPbert_grp6_dim_512_256_128_cw0_5_rank_16_alpha_128_ep10_noES/best_model
/content/EDU-Chunking-RAG/dataset/reduced_fever_data exists: True
/content/EDU-Chunking-RAG/dataset/reduced_fever_data/wiki exists: True
/content/EDU-Chunking-RAG/dataset/reduced_fever_data/paper_dev.jsonl exists: True
/content/EDU-Chunking-RAG/edu_segmenter_linear/MLPbert_grp6_dim_512_256_128_cw0_5_rank_16_alpha_128_ep10_noES/best_model exists: True


In [ ]:
%cd /content/EDU-Chunking-RAG


/content/EDU-Chunking-RAG


In [ ]:
from pathlib import Path

file_path = Path("/content/EDU-Chunking-RAG/com/fever/rag/utils/data_helper.py")
text = file_path.read_text()

# Replace the wrong import with the correct one
text = text.replace(
    "from sympy.printing.pytorch import torch",
    "import torch"
)

file_path.write_text(text)
print("Patched data_helper.py ✅")


Patched data_helper.py ✅


In [ ]:
%cd /content/EDU-Chunking-RAG

# Install a stable version that has `search`
!pip install "qdrant-client==1.7.3"


/content/EDU-Chunking-RAG
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.3/206.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.2/323.2 kB 23.0 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 6.33.1 which is incompatible.
tensorflow 2.19.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.3, but you have protobuf 6.33.1 which is incompatible.
google-ai-generativelanguage 0.6.15 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.2, bu

In [ ]:
from qdrant_client import QdrantClient

client = QdrantClient(":memory:")
print([m for m in dir(client) if "search" in m])


['search', 'search_batch', 'search_groups']


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# 1. Make base folder in Drive
!mkdir -p /content/drive/MyDrive/Experiment_Customedu_fever_rag_runs

# 2. Remove any old/broken link or folder at /content/Experiment_Customedu_fever_rag_runs
!rm -rf /content/Experiment_Customedu_fever_rag_runs

# 3. Create the symlink:
#    LINK:   /content/Experiment_Customedu_fever_rag_runs
#    TARGET: /content/drive/MyDrive/Experiment_Customedu_fever_rag_runs
!ln -s /content/drive/MyDrive/Experiment_Customedu_fever_rag_runs /content/Experiment_Customedu_fever_rag_runs

# 4. Make your run-specific folder (this will actually live in Drive, via the symlink)
!mkdir -p /content/Experiment_Customedu_fever_rag_runs/Experiment_gemma2b_zero_shot

# 5. Check that everything exists
!ls -ld /content/Experiment_Customedu_fever_rag_runs
!ls -ld /content/Experiment_Customedu_fever_rag_runs/Experiment_gemma2b_zero_shot


lrwxrwxrwx 1 root root 58 Nov 28 01:20 /content/Experiment_Customedu_fever_rag_runs -> /content/drive/MyDrive/Experiment_Customedu_fever_rag_runs
drwx------ 2 root root 4096 Nov 28 01:08 /content/Experiment_Customedu_fever_rag_runs/Experiment_gemma2b_zero_shot


In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh



>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:

!pip install -q ollama


In [ ]:
import subprocess, os, time
import urllib.request

# Make sure the binary is on PATH (usually already is, but this is safe)
os.environ["PATH"] += os.pathsep + "/usr/local/bin"

# (Optional but recommended) kill any old server if you re-run this cell
try:
    subprocess.run(["pkill", "-f", "ollama"], check=False)
except Exception:
    pass

# Start Ollama server as a background process
server_proc = subprocess.Popen(
    ["ollama", "serve"],
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL,
)

print("Starting Ollama server...")

# Wait until the server is actually ready
for i in range(30):
    try:
        urllib.request.urlopen("http://127.0.0.1:11434/api/tags", timeout=1)
        print("✅ Ollama is up and running on http://127.0.0.1:11434")
        break
    except Exception:
        time.sleep(1)
else:
    print("❌ Ollama server did not start properly.")

Starting Ollama server...
✅ Ollama is up and running on http://127.0.0.1:11434


In [ ]:
# Check if Ollama is still running
if server_proc.poll() is None:
    print("✅ Ollama server is still running")
else:
    print("❌ Ollama server has stopped")


✅ Ollama server is still running


In [ ]:
import urllib.request

try:
    with urllib.request.urlopen("http://127.0.0.1:11434/api/tags", timeout=2) as r:
        print("✅ Ollama is responding, status:", r.status)
except Exception as e:
    print("❌ Ollama not responding:", e)


✅ Ollama is responding, status: 200


In [ ]:
!ps aux | grep ollama


root        2749  2.0  0.2 1856484 32720 ?       Sl   01:21   0:00 ollama serve
root        2794  0.0  0.0   7376  3484 ?        S    01:21   0:00 /bin/bash -c ps aux | grep ollama
root        2796  0.0  0.0   6484  2416 ?        S    01:21   0:00 grep ollama


In [ ]:
!ollama pull gemma:2b

In [ ]:
%cd /content/EDU-Chunking-RAG

# Make sure the run folder exists (this is actually inside Drive via the symlink)
!mkdir -p /content/Experiment_Customedu_fever_rag_runs/Experiment_gemma2b_zero_shot

!PYTHONUNBUFFERED=1 python -u -m com.fever.rag.runner \
  --llm_name "gemma:2b" \
  --few_shot_examples 0 \
  --temperature 0 \
  --qdrant_in_memory \
  --embedding_model_name sentence-transformers/all-MiniLM-L6-v2 \
  --wiki_dir dataset/reduced_fever_data/wiki \
  --claim_file_path dataset/reduced_fever_data/paper_test.jsonl \
  --chunker_type custom_edu \
  --chunking_overlap 0 \
  --max_tokens 150 \
  --retrieval_strategy top_k \
  --model_path "$MODEL_PATH" \
  --top_k 20 \
  --output_dir "/content/Experiment_Customedu_fever_rag_runs/Experiment_gemma2b_zero_shot" \
  > "/content/Experiment_Customedu_fever_rag_runs/Experiment_gemma2b_zero_shot/gemma2b_customedu_zero_shot.log" 2>&1


/content/EDU-Chunking-RAG
